In [1]:
import cv2
import json
import numpy as np
import os
import random
from PIL import ImageFont, ImageDraw, Image
from preprocess.restructure._settings import (
    det_yolo_to_aihub_mapper, seg_yolo_to_aihub_mapper,)

In [2]:
fontpath = 'misc/NanumGothicBold.ttf'
font = ImageFont.truetype(fontpath, 30)

def single_image(image_file_path):
    'image 불러오기'
    image_array = np.fromfile(image_file_path)
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    return image

image_selection_map = {
    'CCTV': 'C', '블랙박스': 'B', '주간': 'D', '야간': 'N', '맑음': 'F', '우천': 'R', '실증': '1', '연출': '0'
}

def random_image_aihub(cam='CCTV', time_='주간', weather='맑음', real='실증', image_file_name=None):
    '''
    cam: CCTV 'C', 블랙박스 'B'\n
    time_: 주간 'D', 야간 'N'\n
    weather: 맑음 'F', 우천 'R'\n
    real: 실증 1, 연출 0\n\n
    파일 이름 예시: 'data/images/train/0000000_000_000_{cam}_{time_}_{weather}_{real}\n
    파일 이름 명시 가능\n\n
    image, json_file_path, image_file_path, json_file_path 반환
    '''
    # 파일 경로 설정
    image_parent_path = os.path.join(os.getcwd(), 'data', 'images', 'train')
    if not image_file_name:
        # 인자 mapping
        args = [cam, time_, weather, real]
        args = [image_selection_map[arg] for arg in args]

        all_image_file_names = os.listdir(image_parent_path)
        category = '_'.join(args)
        image_file_name = random.choice(list(filter(lambda file: category in file, all_image_file_names)))
    image_file_path = os.path.join(image_parent_path, image_file_name)
    json_file_path = image_file_path.replace('images', 'labels_json').replace('.jpg', '.json').replace('.jpeg', '.json')

    # 이미지 파일 읽기
    image = single_image(image_file_path)

    # json 파일 읽기
    with open(json_file_path,'r') as json_file:
        data = json.load(json_file)

    print(image_file_name)

    return image, data, image_file_path, json_file_path

# 테스트 이미지 json
_, __, image_file_path, json_file_path = random_image_aihub()
print(image_file_path, json_file_path, sep='\n')

F000011_005_3316_C_D_F_1.jpg
c:\Users\jinmo\Documents\GitHub\tp2\data\images\train\F000011_005_3316_C_D_F_1.jpg
c:\Users\jinmo\Documents\GitHub\tp2\data\labels_json\train\F000011_005_3316_C_D_F_1.json


In [3]:
# # 파일 경로 설정
# json_file_path = './data/라벨링data/VL1/CCTV/야간/맑음/실증/C000003_002_0204_C_N_F_0.json'
# image_file_path = './data/원천data/VS1/CCTV/야간/맑음/실증/C000003_002_0204_C_N_F_0.jpg'
# image_array = np.fromfile(image_file_path)

# # 이미지 파일 읽기
# image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
# # image = cv2.imread('./T006222_006_0180_C_D_F_0.jpg') # 이미지 경로 수정 후 파일 로드
# # 라벨링 파일(json) 읽기
# with open(json_file_path,'r') as json_file:
#     data = json.load(json_file)

In [4]:
# # 라벨링 data에서 읽어온 좌표 적용 제대로 안됨
# # image = cv2.imread('./C000003_002_0204_C_N_F_0.jpg')
# # image = np.zeros((2000,2000,3))

# image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
# polygon_point_test = np.array(data['annotations']['environment'][2]['points'],np.float32)
# polygon_point_test = polygon_point_test.reshape((-1,2))
# polygon_point_test = polygon_point_test.astype(np.int32)
# cv2.polylines(image, [polygon_point_test], isClosed=True, color = (0,0,255), thickness=2)
# cv2.imshow('Labeled Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [5]:
# #문제 해결 "x,y 좌표 수정 후 적용 "
# # image = cv2.imread('./C000003_002_0204_C_N_F_0.jpg')
# # image = np.zeros((2000,2000,3))

# image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
# polygon_point_test = data['annotations']['environment'][2]['points']
# swapped_polygon = [[y,x] for x, y in polygon_point_test]
# polygon_point_test = np.array(swapped_polygon,np.float32)
# polygon_point_test = polygon_point_test.reshape((-1,2))
# polygon_point_test = polygon_point_test.astype(np.int32)
# cv2.polylines(image, [polygon_point_test], isClosed=True, color = (0,0,255), thickness=2)
# cv2.imshow('Labeled Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [6]:
# # 모든 polygon 데이터 시각화
# # image = cv2.imread('./C000003_002_0204_C_N_F_0.jpg')
# image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
# for label_info in data['annotations']['environment']:
#     label_type = label_info['shape_type']
#     # if label_type == 'bbox':
#     #     x, y, w, h = map(int, label_info['points'])
#     #     cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0),2)
#     if label_type == 'polygon':
#         points = label_info['points']
#         points = [[y,x] for x, y in points]
#         polygon_points = np.array(points, np.float32)
#         polygon_points = polygon_points.reshape((-1,2))
#         polygon_points = polygon_points.astype(np.int32)
#         cv2.polylines(image, [polygon_points], isClosed=True, color= (255,0,0), thickness=2)
# cv2.imshow('Labeled Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [7]:
# # Area Code 추가
# # image = cv2.imread('./C000003_002_0204_C_N_F_0.jpg')
# image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)
# for label_info in data['annotations']['environment']:
#     label_type = label_info['shape_type']
#     # if label_type == 'bbox':
#     #     x, y, w, h = map(int, label_info['points'])
#     #     cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0),2)
#     if label_type == 'polygon':
#         points = label_info['points']
#         points = [[y,x] for x, y in points]
#         polygon_points = np.array(points, np.float32)
#         polygon_points = polygon_points.reshape((-1,2))
#         polygon_points = polygon_points.astype(np.int32)
#         area_code = label_info['area_code']
#         cv2.polylines(image, [polygon_points], isClosed=True, color= (255,0,0), thickness=2)
#         cv2.putText(image, str(area_code), (polygon_points[1, 0], polygon_points[1, 1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
# cv2.imshow('Labeled Image', image)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [8]:
# AI-Hub json 파일로부터 annotation 그리는 함수
def draw_korean_texts(image, *texts_info):
    '한글 그려주는 함수. 이미지 array로 반환'
    image = Image.fromarray(image)
    draw = ImageDraw.Draw(image)
    for text_info in texts_info:
        draw.text(*text_info, font)

    return np.array(image)

def draw_aihub_annotations(image, data):
    'AI-Hub json 파일로 annotation 표기'
    # texts_info = []
    # env
    for label_info in data['annotations']['environment']:
        points = label_info['points']
        points = [[y,x] for x, y in points]
        polygon_points = np.array(points, np.float32)
        polygon_points = polygon_points.reshape((-1,2))
        polygon_points = polygon_points.astype(np.int32)
        area_code = label_info['area_code']
        cv2.polylines(image, [polygon_points], isClosed=True, color= (255,0,0), thickness=2)
        # texts_info.append(((polygon_points[1, 0], polygon_points[1, 1]), area_code, (255,0,0)))
        cv2.putText(image, str(area_code), (polygon_points[1, 0], polygon_points[1, 1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    # PM
    for label_info in data['annotations']['PM']:
        x, y, w, h = map(int, label_info['points'])
        pm_code = label_info['PM_code']
        cv2.rectangle(image, (x, y), (x+w, y+h), (0,255,0),2)
        # texts_info.append(((x,y), pm_code, (0,255,0)))
        cv2.putText(image, pm_code, (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    # text 그리기
    # image = draw_korean_texts(image, *texts_info)
    image = cv2.resize(image, (1920,1080)) # 간혹 너비 3072 있음. 1920으로 통일
    cv2.imshow('Labeled Image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [9]:
# image, data, _, __ = random_image_aihub()
image, data, _, __ = random_image_aihub(image_file_name='T008037_067_0096_C_D_F_0.jpg')
draw_aihub_annotations(image, data)

T008037_067_0096_C_D_F_0.jpg


In [9]:
# yolo 형식 .txt로부터 annotation 그리는 함수
def draw_yolo_annotations(image_file_path, det_txt_path, seg_txt_path):
    'YOLO .txt로부터 annotation 표기'
    image = single_image(image_file_path)

    # 이미지의 너비와 높이를 가져옵니다.
    image_height, image_width = image.shape[:2]

    # YOLO 형식의 .txt 파일로부터 annotation 정보를 읽어옵니다.
    annotations_det = read_yolo_bbox(det_txt_path, image_width, image_height)
    annotations_seg = read_yolo_polygons(seg_txt_path, image_width, image_height)

    # 이미지에 bounding box와 polygon을 그려줍니다.
    draw_annotations(image, annotations_det, annotations_seg)

    # 이미지 사이즈 통일
    image = cv2.resize(image, (1920,1080)) # 간혹 너비 3072 있음. 1920으로 통일

    # 결과 이미지를 표시합니다.
    cv2.imshow("Annotated Image", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# YOLO 형식의 .txt 파일로부터 bounding box 정보를 읽어옵니다.
def read_yolo_bbox(file_path, image_width, image_height):
    annotations = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = det_yolo_to_aihub_mapper[int(parts[0])]
            x_center, y_center, bbox_width, bbox_height = map(float, parts[1:])
            x_min = int((x_center - bbox_width / 2) * image_width)
            y_min = int((y_center - bbox_height / 2) * image_height)
            x_max = int((x_center + bbox_width / 2) * image_width)
            y_max = int((y_center + bbox_height / 2) * image_height)
            annotations.append((class_id, x_min, y_min, x_max, y_max))
    return annotations

# YOLO 형식의 .txt 파일로부터 polygon 정보를 읽어옵니다.
def read_yolo_polygons(file_path, image_width, image_height):
    polygons = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        for line in lines:
            parts = line.strip().split()
            class_id = seg_yolo_to_aihub_mapper[int(parts[0])]
            numbers = [float(p) for p in parts[1:]]
            yolo_points = [numbers[i:i+2] for i in range(0,len(numbers),2)]
            pixel_points = [(int(point[0] * image_height), int(point[1] * image_width)) for point in yolo_points]
            pixel_points_xy = list(map(lambda point: [point[1], point[0]], pixel_points))
            polygons.append((class_id, pixel_points_xy))
    return polygons

# 이미지에 bounding box와 polygon을 그려주는 함수
def draw_annotations(image, annotations_det, annotations_seg):
    for bbox in annotations_det:
        pm_code, x_min, y_min, x_max, y_max = bbox
        pm_code = int(pm_code)
        color = (0, 255, 0)  # BGR color (green)
        thickness = 2
        cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color, thickness)
        cv2.putText(image, str(pm_code), (x_min, y_min), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    for polygon in annotations_seg:
        area_code, points = polygon
        color = (255, 0, 0)  # BGR color (green)
        thickness = 2
        points = np.array(points, np.int32)
        points = points.reshape((-1, 2))
        cv2.polylines(image, [points], isClosed=True, color=color, thickness=thickness)
        cv2.putText(image, str(area_code), (points[1, 0], points[1, 1]), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

In [10]:
# 이미지 파일 경로와 YOLO 형식의 .txt 파일 경로
image_file_path = "data/images/train/T007931_000_0070_B_D_F_0.jpg"
det_txt_path = "data/labels/train/T007931_000_0070_B_D_F_0.txt"
seg_txt_path = "data/labels_seg/train/T007931_000_0070_B_D_F_0.txt"

def random_paths_yolo(cam='CCTV', time_='주간', weather='맑음', real='실증', det_path='labels_det', seg_path='labels_seg'):
    '''
    cam: CCTV 'C', 블랙박스 'B'\n
    time_: 주간 'D', 야간 'N'\n
    weather: 맑음 'F', 우천 'R'\n
    real: 실증 1, 연출 0\n\n
    파일 이름 예시: 'data/train/images/0000000_000_000_{cam}_{time_}_{weather}_{real}\n\n
    image_file_path, det_txt_path, seg_txt_path 반환
    '''
    # 인자 mapping
    args = [cam, time_, weather, real]
    args = [image_selection_map[arg] for arg in args]

    # 파일 경로 설정
    image_parent_path = os.path.join(os.getcwd(), 'data', 'images', 'train')
    all_image_file_names = os.listdir(image_parent_path)
    category = '_'.join(args)
    image_file_name = random.choice(list(filter(lambda file: category in file, all_image_file_names)))
    image_file_path = os.path.join(image_parent_path, image_file_name)
    det_file_path = image_file_path.replace('images', det_path).replace('.jpg', '.txt').replace('.jpeg', '.txt')
    seg_file_path = det_file_path.replace(det_path, seg_path)

    print(image_file_path)

    return image_file_path, det_file_path, seg_file_path

In [31]:
draw_yolo_annotations(*random_paths_yolo(det_path='labels'))

c:\Users\jinmo\Documents\GitHub\tp2\data\images\train\T012266_000_0063_C_D_F_1.jpg


In [8]:
draw_yolo_annotations(image_file_path, det_txt_path, seg_txt_path)